In [1]:
import numpy
import time
import pickle
import operator
from __future__ import print_function
from IPython.display import display

In [2]:

def load_file(filepath):
    """Reads a file into a list of phrases. Each phrase in the file must be separated with a new line character '\n' 
    Args:
        filepath (str): the relevant filepath of the file
    Returns:
        phrases (list(str)): A list with all the phrases 
    """
    with open(filepath, 'r') as f:
        phrases = f.readlines()
    return phrases

src_file = load_file('data/file.de')
trg_file = load_file('data/file.en')
aligned_file = load_file('data/file.aligned')

## Extract phrases

In [12]:


def add_freq_dict(dictionary, key, value):
    """Increases the frequency of an phrase/item in the dictionary by 1, or adds it with a frequency of 1 if it doesn't exist yet
    Args:
        dictionary (dict): a dictionary that holds the frequency of an a phrase (item)
        item (str): a string whose frequency will be increased
    """
    if key in dictionary:
            dictionary[key][0] = [x+y for x,y in zip(dictionary[key][0] , value[0])]
            dictionary[key][1] = [x+y for x,y in zip(dictionary[key][1] , value[1])]
    else:
        dictionary[key] = value


def extract(trg_start, trg_end, src_start, src_end, alignments, trg_aligned, trg_len):    
    """ Checks if the subphrases of the source and target language are a consisnte pair, 
        assuming that no word in the source language is aligned with a word of the target language  outside the boundaries
    Args:
        trg_start (int): The target subphrase's first word position
        trg_end (int): The target subphrase's last word position
        src_start (int): The source subphrase's first word position
        src_end (int): The source subphrase's last word position
        alignments (list(tuple(int))): The alignment between the 
        trg_aligned (set(str)): A set with all the target language words that are aligned with at least one source word 
        trg_len (int): The number of words of the target language phrase
        
    Returns:
        E (list(tuple(str))): A list with all of the extracted phrase pairs (f,e). This is the base (f,e) specified by the arguement and pairs that have a words appended on 'e' that are unaligned 
        A (list(list(tuple(int)))): A list with the alignment of the respective pair at E (A[i] has the alignments of E[i]) 

    """
    if trg_end < 0 :
        return [], []
    
    A = []
    
    a = []
    
    for trg_word, src_word  in alignments:
        if trg_start <= trg_word <= trg_end:
            
            if (src_word < src_start or src_word > src_end):  
                return [], []
            else:
                
                a.append((trg_word - trg_start, src_word - src_start))

    E = []
    trg_s = trg_start
    
    while True:
        trg_e = trg_end
        while True:
            E.append([src_start, src_end, trg_s, trg_e])
            
            A.append(a)
             
            return E, A
            trg_e += 1
            
            if trg_e in trg_aligned or trg_e == len(trg_words):
                break 

        trg_s -= 1
        
        if trg_s in trg_aligned or trg_s < 0:
            break
            
        # if an unaligned word was added at the beginning, increase the alignments positions by 1 
       
        a = [(i[0]+1,i[1]) for i in a]
        
    return E, A


def extract_phrases(src_words, trg_words, alignments, cutoff):
    """
    Args:
        src_words (list(str)):
        trg_words (list(str)):
        alignments (list(tuple(int))):
        cutoff (int): 
        
    Returns:
        extracted_phrases (list(tuple(str))):
        extracted_alignments (list(tuple(int))):
    """
    
    if cutoff == -1:
        cutoff = len(src_words)
        
        
    trg_aligned = set()
    for (trg,_) in alignments:
        trg_aligned.add(trg)
    
    
    extracted_phrases = []
    extracted_alignments = []
    # for every possible 
    for src_start in range(len(src_words)):
        for src_end in range(src_start, min(src_start + cutoff, len(src_words))):

            #Find the min and max position of the target words that the source phrase is aligned to
            trg_start = len(trg_words) - 1
            trg_end = - 1
            for  (trg,src) in alignments: 
                if src_start <= src <= src_end:
                    trg_start = min(trg, trg_start)
                    trg_end = max(trg, trg_end)
                    
            
            if(trg_end - trg_start > cutoff - 1):
                continue

            phrase_pairs, A = extract(trg_start, trg_end, src_start, src_end, alignments, trg_aligned, len(trg_words))
            
            if (phrase_pairs):
                extracted_phrases.extend(phrase_pairs)
                for a in A:
                    extracted_alignments.append(a);
            
    return extracted_phrases, extracted_alignments




def count_reorderings(src_words, trg_words, A, word_based = False):
    """
    
    """
    temp = {}
    
    pp, _ = extract_phrases(src_words, trg_words, A, 7)
    
    if word_based:
        next_iterator = A
    else:
        next_iterator = pp

    for phrase1  in pp:
        [src_s, src_e, trg_s, trg_e] = phrase1
        
        m, s, dr, dl = 0,0,0,0 #left-to-right reordering counters
        m2, s2, dr2, dl2 = 0,0,0,0 #right-to-left reordering counters

        for a in next_iterator:
            
            if word_based:
                [next_src_s, next_src_e, next_trg_s, next_trg_e] = [a[1], a[1], a[0], a[0]]
            else:
                [next_src_s, next_src_e, next_trg_s, next_trg_e] = a
                
                

            # left to right
            #if it is the next word, check if monotone/swap/discontinues to the right
            if next_trg_s == trg_e + 1:
                if next_src_s == src_e + 1:
                    m+=1
                elif src_s == next_src_e + 1:
                    s+=1
                else: 
                    if next_src_s > src_e:
                        dr += 1
                    elif next_src_e < src_s:
                        dl +=1

            # right to left 
             #if it is the next word, check if monotone/swap/discontinues to the right
            if  next_trg_e == trg_s - 1:
                if next_src_e == src_s - 1:
                    m2+=1
                elif next_src_s == src_e + 1:
                    s2+=1
                else:
                    if next_src_s > src_e:
                        dl2 += 1
                    elif next_src_e < src_s:
                        dr2 += 1


        pair = (" ".join(trg_words[trg_s:trg_e+1]) , " ".join(src_words[src_s:src_e+1]))        

        add_freq_dict(temp, pair, [[m,s,dr,dl] , [m2,s2,dr2,dl2]])

    
    return temp



        



In [19]:
phrase_pairs = dict() 

start_time = time.time()
    
for i in range(len(src_file)):
    src_words = src_file[i].split()
    trg_words = trg_file[i].split()        
    alignments = [[int(a) for a in alignment.split('-')] for alignment in aligned_file[i].split()] #convert to list of int pairs
    alignments = [(al[1], al[0]) for al in alignments] #reverse their order so that they are (trg,src) instead of (src,trg)
    
    this_phrase_pairs = count_reorderings(src_words, trg_words, alignments)
    
    for key,value in this_phrase_pairs.items():
        add_freq_dict(phrase_pairs, key, value)
        
    #show real time progress
    if ((i+1) % (len(src_file) / 100) == 0):
        print(' \r%d / %d (%ds)'%(i+1,len(src_file), time.time() - start_time), end = '')

print()
print('Done!!!')
print('Total duration: %ds'%(time.time() - start_time))

print("#unique en phrases: %d"%(len(phrase_pairs)))

50000 / 50000 (110s)                                                                                               
Done!!!
Total duration: 110s
#unique en phrases: 2277463


In [ ]:
for key in phrase_pairs.keys()[10:20]:
    print(key, phrase_pairs[key])

In [20]:
with open('data/reordering_pairs_phrase.pkl', 'wb') as file:
    pickle.dump(phrase_pairs, file)

# with open('data/reordering_pairs_phrase.pkl', 'rb') as file:
#     phrase_pairs = pickle.load(file)

In [57]:
#convert counts to probabilities
for key, value in phrase_pairs.items():
    
    sum_ = numpy.sum(value[0])
    if sum_ > 0:
        for i in range(4):
            phrase_pairs[key][0][i] /= 1.0 * sum_

    sum_ = numpy.sum(value[1])
    if sum_ > 0:
        for i in range(4):
            phrase_pairs[key][1][i] /= 1.0* sum_

In [58]:
for key in phrase_pairs.keys()[10:20]:
    print(key, phrase_pairs[key])

('in its relations with the', 'in seinen beziehungen zu den') [[0.2, 0.0, 0.8, 0.0], [0.0, 0.0, 0.0, 1.0]]
('be tackled separately', 'gesonderter form durchgef\xc3\xbchrt werden') [[0, 0, 0, 0], [0.0, 1.0, 0.0, 0.0]]
('other relevant', 'weitere zweckm\xc3\xa4\xc3\x9fige') [[1.0, 0.0, 0.0, 0.0], [0, 0, 0, 0]]
('even', 'auch dann') [[0.0, 0.0, 0.0, 1.0], [0.0, 0.25, 0.75, 0.0]]
('a strong voice in', 'einer starken stimme in') [[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0]]
('is dependent on aid , as', 'auf f\xc3\xb6rderungen angewiesen ist , da') [[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0]]
('key aspects', 'wesentliche aspekte') [[0, 0, 0, 0], [1.0, 0.0, 0.0, 0.0]]
('the duty', 'die aufgabe') [[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0]]
('social protection', 'ausreichenden sozialschutz') [[0.0, 0.0, 0.0, 1.0], [0.5, 0.0, 0.5, 0.0]]
('guarantee that', 'es garantien gibt , da\xc3\x9f') [[0.18181818181818182, 0.0, 0.0, 0.8181818181818182], [0.25, 0.0, 0.75, 0.0]]


In [ ]:
# en = "Peter has scored a goal ."
# de = "Peter hat ein Tor geschossen ."
# A = "0-0 1-1 4-2 2-3 3-4 5-5"
# A = [[int(a) for a in alignment.split('-')] for alignment in A.split()] #convert to list of int pairs 
# src_words = de.split();
# trg_words = en.split();
# alignment_matrix = numpy.zeros((len(src_words),len(trg_words)), dtype=int)
# n = len(src_words) - 1;
# for (i,j) in A: 
#     alignment_matrix[n - i,j] = 1


# A = [(al[1], al[0]) for al in A] #reverse their order so that they are (trg,src) instead of (src,trg)
   
# phrase_pairs = {}
    
# this_phrase_pairs = count_reorderings(src_words, trg_words, A, False)

# for key, value in this_phrase_pairs.items():
#     add_freq_dict(phrase_pairs, key, value)


# print(alignment_matrix) 
# display(phrase_pairs)

In [ ]:
phrase_pairs = dict() 

start_time = time.time()
    
for i in range(len(src_file)):
    src_words = src_file[i].split()
    trg_words = trg_file[i].split()        
    alignments = [[int(a) for a in alignment.split('-')] for alignment in aligned_file[i].split()] #convert to list of int pairs
    alignments = [(al[1], al[0]) for al in alignments] #reverse their order so that they are (trg,src) instead of (src,trg)
    
    pp, pp_a = extract_phrases(src_words, trg_words, alignments, 7)
    
    for (phrase, phrase_alignments) in zip(pp,a):
        phrase_src_words = src_words[phrase[0] : phrase[1] + 1]
        phrase_trg_words = trg_words[phrase[2] : phrase[3] + 1]
        
        this_phrase_pairs = count_reorderings(phrase_src_words, phrase_trg_words, phrase_alignments)
    
        for key,value in this_phrase_pairs.items():
            add_freq_dict(phrase_pairs, key, value)

    #show real time progress
    if ((i+1) % (len(src_file) / 100) == 0):
        print(' \r%d / %d (%ds)'%(i+1,len(src_file), time.time() - start_time), end = '')

print()
print('Done!!!')
print('Total duration: %ds'%(time.time() - start_time))

print("#unique en phrases: %d"%(len(phrase_pairs)))

In [23]:
a = [1,2,3,4]

a[0:2]

[1, 2]